# Tree Segmentation Example
This is a simple example of what the PyCrown package can do: from pre-calculated rasters (CHM, DSM and DTM) and a height-normalized 3D LiDAR point cloud, individual trees can be segmented.
Outputs are shapefiles of tree top locations, crown shapes as well as a .LAS-file containing classified trees.

## Start with importing the modules

In [1]:
import sys
from datetime import datetime
from pycrown import PyCrown

## Set input files
Specify the file locations for the CHM, DSM, DTM and the LiDAR point cloud.
The latter is only needed if the point cloud should be classified into individual trees.

In [2]:
F_CHM = 'data/CHM.tif'
F_DTM = 'data/DTM.tif'
F_DSM = 'data/DSM.tif'
F_LAS = 'data/POINTS.las'

## Initialize an instance of PyCrown

In [3]:
PC = PyCrown(F_CHM, F_DTM, F_DSM, F_LAS, outpath='result')

## Clip all input data to new bounding box.

In [4]:
PC.clip_data_to_bbox((1802150, 1802408, 5467305, 5467480))

## Smooth CHM
A 5x5m block median filter is used (set circular=True to enable a disc-shaped window).

In [5]:
PC.filter_chm(5, ws_in_pixels=True, circular=False)

## Tree Detection with local maximum filter

In [6]:
PC.tree_detection(PC.chm, ws=5, hmin=16.)

## Clip trees to bounding box 
(no trees on image edge)
original extent: 1802140, 1802418, 5467295, 5467490    

In [7]:
PC.clip_trees_to_bbox(bbox=(1802160, 1802400, 5467315, 5467470))

## Crown Delineation

In [8]:
PC.crown_delineation(algorithm='dalponteCIRC_numba', th_tree=15.,
                     th_seed=0.7, th_crown=0.55, max_crown=10.)

Tree crowns delineation: 0.007s


## (Optional) Correct tree tops on steep terrain

In [9]:
PC.correct_tree_tops()

Number of trees: 128
Tree tops corrected: 9
Tree tops corrected: 7.03125%
DSM correction: 5
COM correction: 4


## Calculate tree height and elevation

In [10]:
PC.get_tree_height_elevation(loc='top')
PC.get_tree_height_elevation(loc='top_cor')

## Screen small trees

In [11]:
PC.screen_small_trees(hmin=20., loc='top')

## Convert raster crowns to polygons

In [12]:
PC.crowns_to_polys_raster()
PC.crowns_to_polys_smooth(store_las=True)

Converting LAS point cloud to shapely points
Converting raster crowns to shapely polygons
Attach LiDAR points to corresponding crowns
Create convex hull around first return points
Classifying point cloud


## Check that all geometries are valid

In [13]:
PC.quality_control()

## Print out number of trees

In [14]:
print(f"Number of trees detected: {len(PC.trees)}")

Number of trees detected: 115


## Export results

In [15]:
PC.export_raster(PC.chm, PC.outpath / 'chm.tif', 'CHM')
PC.export_tree_locations(loc='top')
PC.export_tree_locations(loc='top_cor')
PC.export_tree_crowns(crowntype='crown_poly_raster')
PC.export_tree_crowns(crowntype='crown_poly_smooth')